In [1]:
from itertools import product
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot
pd.options.display.float_format = '${:,.2f}'.format

In [2]:
df = pd.read_csv('./data.csv')
df.shape

(17, 3)

In [3]:
df.head()

,height,class,price
0,65,2,3190
1,65,3,3400
2,70,4,3863
3,85,2,4105
4,85,3,4644


In [4]:
sc = StandardScaler()

In [5]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [6]:
X = sc.fit_transform(X)

/home/greg/.virtualenvs/ga/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/greg/.virtualenvs/ga/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [12]:
r = svm.SVR(gamma='auto')

In [13]:
gs = GridSearchCV(r,{'C': [1,10,100,1000,10000,100000], 'kernel': ['poly', 'rbf', 'sigmoid']}, cv=3)

In [14]:
gs.fit(X, y)

/home/greg/.virtualenvs/ga/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1, 10, 100, 1000, 10000, 100000], 'kernel': ['poly', 'rbf', 'sigmoid']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [15]:
gs.best_estimator_

SVR(C=1000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='sigmoid', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [16]:
gs.best_estimator_.score(X, y)

0.5831345772171546

#### Make predictions against full parameter sweeps

In [17]:
# make class and heights
h = np.arange(65,145,5)
c = np.arange(1,15,1)

In [18]:
# create matrix of all possible input features
feat_pred = pd.DataFrame(list(product(h,c)), columns=['height', 'class'])

In [19]:
# create predictions against the standard scaled versions of those features
tgt_pred = pd.DataFrame(gs.best_estimator_.predict(sc.transform(feat_pred)), columns=['price'])

/home/greg/.virtualenvs/ga/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  


In [20]:
# join those shits
df_pred = feat_pred.join(tgt_pred)

In [21]:
out = df_pred.pivot(index='height', columns='class')
out

price                                                              \
class         1         2         3         4         5         6         7    
height                                                                         
65     $2,287.81 $2,731.01 $4,300.83 $4,977.32 $4,791.22 $4,332.28 $3,803.81   
70     $2,532.19 $2,789.23 $4,172.95 $4,809.86 $4,552.18 $4,048.08 $3,572.22   
75     $2,893.40 $2,987.66 $4,142.43 $4,700.02 $4,367.59 $3,881.49 $3,515.53   
80     $3,369.40 $3,339.81 $4,237.68 $4,688.55 $4,317.99 $3,917.51 $3,672.24   
85     $3,941.11 $3,821.79 $4,467.43 $4,828.62 $4,482.67 $4,195.31 $4,046.04   
90     $4,554.83 $4,357.36 $4,803.98 $5,154.90 $4,880.32 $4,685.38 $4,595.00   
95     $5,111.12 $4,831.74 $5,180.17 $5,636.09 $5,443.29 $5,294.61 $5,226.58   
100    $5,496.16 $5,147.12 $5,511.97 $6,171.50 $6,056.89 $5,907.82 $5,828.78   
105    $5,646.88 $5,276.90 $5,734.17 $6,647.43 $6,624.01 $6,445.17 $6,324.83   
110    $5,586.86 $5,258.22 $5,821.50 $6,990.73 $7,093.56 $6,885.15 $6,699.09   
115    $5,397.08 $5,144.03 $5,784.35 $7,180.28 $7,447.84 $7,242.61 $6,980.58   
120    $5,156.67 $4,974.10 $5,651.78 $7,231.12 $7,685.52 $7,535.70 $7,212.02   
125    $4,910.58 $4,773.46 $5,456.34 $7,174.01 $7,816.02 $7,769.70 $7,425.71   
130    $4,674.68 $4,558.38 $5,226.06 $7,041.11 $7,857.18 $7,941.18 $7,631.76   
135    $4,451.65 $4,340.20 $4,981.97 $6,859.15 $7,830.44 $8,049.27 $7,820.62   
140    $4,240.77 $4,126.83 $4,738.39 $6,647.98 $7,755.78 $8,100.52 $7,976.44   

                                                                              
class         8         9         10        11        12        13        14  
height                                                                        
65     $3,381.93 $3,137.76 $3,023.37 $2,975.24 $2,955.91 $2,948.30 $2,945.33  
70     $3,265.87 $3,113.50 $3,047.56 $3,020.78 $3,010.18 $3,006.03 $3,004.41  
75     $3,317.80 $3,228.80 $3,192.02 $3,177.38 $3,171.62 $3,169.38 $3,168.50  
80     $3,555.28 $3,505.74 $3,485.80 $3,477.94 $3,474.87 $3,473.67 $3,473.20  
85     $3,980.57 $3,953.84 $3,943.25 $3,939.10 $3,937.48 $3,936.85 $3,936.60  
90     $4,557.48 $4,542.51 $4,536.64 $4,534.34 $4,533.45 $4,533.10 $4,532.96  
95     $5,198.58 $5,187.47 $5,183.11 $5,181.41 $5,180.74 $5,180.49 $5,180.39  
100    $5,794.08 $5,779.92 $5,774.30 $5,772.10 $5,771.25 $5,770.91 $5,770.78  
105    $6,266.41 $6,241.56 $6,231.55 $6,227.60 $6,226.06 $6,225.46 $6,225.22  
110    $6,597.39 $6,551.91 $6,533.20 $6,525.76 $6,522.84 $6,521.71 $6,521.26  
115    $6,814.18 $6,734.46 $6,700.70 $6,687.13 $6,681.77 $6,679.68 $6,678.87  
120    $6,962.67 $6,830.99 $6,772.73 $6,748.86 $6,739.39 $6,735.67 $6,734.23  
125    $7,088.39 $6,884.69 $6,788.39 $6,747.78 $6,731.47 $6,725.05 $6,722.54  
130    $7,225.40 $6,933.30 $6,781.22 $6,714.13 $6,686.67 $6,675.77 $6,671.50  
135    $7,388.93 $7,006.46 $6,778.93 $6,671.45 $6,626.10 $6,607.87 $6,600.69  
140    $7,572.67 $7,122.23 $6,804.45 $6,638.59 $6,565.20 $6,535.10 $6,523.15

In [22]:
# write to file of course
out.to_excel('./out.xlsx', sheet_name='predictions', float_format='%.2f')